<a href="https://colab.research.google.com/github/l-pommeret/EDT-LOGOS1/blob/main/Po%C3%A8teGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dans ce premier notebook, nous apprenons à faire un modèle simple dont la tâche est de générer des vers français.
Ici nous n'allons pas nous embarasser de complexité : nous utilisons toutes les fonctions de très haut niveau que les bibliothèques de HuggingFace (`transformers`, `datasets`) nous offrent.

Ici l'idée est de montrer une idée simple pour comprendre l'intuition derrière un LLM de type GPT.

# Run

## Création du dataset

In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Chargement du dataset
ds = load_dataset("manu/french_poetry")

# Fonction pour séparer les vers et les filtrer
def split_and_filter_verses(poem):
    return [
        verse.strip()
        for verse in poem['text'].split('\n')
        if verse.strip() and not any(verse.strip().startswith(prefix) for prefix in ["Poésie :", "Titre :", "Poète :"])
        and 20 <= len(verse.strip()) <= 60
    ]

# Préparation du dataset
all_verses = []
for poem in ds['train']:
    all_verses.extend(split_and_filter_verses(poem))

print(f"Nombre total de vers après filtrage : {len(all_verses)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/486 [00:00<?, ?B/s]

(…)-00000-of-00001-927b7145806656e5.parquet:   0%|          | 0.00/2.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Nombre total de vers après filtrage : 65082


In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['title', 'poet', 'text', 'link', 'id'],
        num_rows: 1821
    })
})


In [ ]:
class CharacterTokenizer:
    def __init__(self):
        self.vocab = {'<PAD>': 0, '<UNK>': 1}
        self.ids_to_tokens = {0: '<PAD>', 1: '<UNK>'}
        self.next_id = 2

    def build_vocab(self, texts):
        for text in texts:
            for char in text:
                if char not in self.vocab:
                    self.vocab[char] = self.next_id
                    self.ids_to_tokens[self.next_id] = char
                    self.next_id += 1
        print(f"Taille du vocabulaire: {len(self.vocab)}")

    def __len__(self):
        return len(self.vocab)

    def tokenize(self, text):
        return list(text)

    def convert_tokens_to_ids(self, tokens):
        return [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]

    def convert_ids_to_tokens(self, ids):
        return [self.ids_to_tokens[id] for id in ids]

    def encode(self, text):
        return self.convert_tokens_to_ids(self.tokenize(text))

    def decode(self, ids):
        return ''.join(self.convert_ids_to_tokens(ids))

    @property
    def vocab_size(self):
        return len(self.vocab)

In [ ]:
class VerseDataset(Dataset):
    def __init__(self, verses, tokenizer, max_length):
        self.verses = verses
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.verses)

    def __getitem__(self, idx):
        verse = self.verses[idx]
        encoded = self.tokenizer.encode(verse)
        if len(encoded) > self.max_length:
            encoded = encoded[:self.max_length]
        else:
            encoded += [self.tokenizer.vocab['<PAD>']] * (self.max_length - len(encoded))

        # Retourner un dictionnaire avec 'input_ids' et 'labels'
        return {
            'input_ids': torch.tensor(encoded),
            'labels': torch.tensor(encoded)
        }

In [ ]:
# Création du tokenizer
tokenizer = CharacterTokenizer()

# Construction du vocabulaire à partir de tous les vers
tokenizer.build_vocab(all_verses)

import random
from torch.utils.data import random_split

# Mélange aléatoire des versets
random.shuffle(all_verses)

# Calcul de la taille du test dataset (5% des données)
test_size = int(0.05 * len(all_verses))
train_size = len(all_verses) - test_size

# Création des datasets
train_dataset = VerseDataset(all_verses[:train_size], tokenizer, max_length=60)
test_dataset = VerseDataset(all_verses[train_size:], tokenizer, max_length=60)

# Affichage des tailles des datasets
print(f"Taille du train dataset : {len(train_dataset)}")
print(f"Taille du test dataset : {len(test_dataset)}")

Taille du vocabulaire: 114
Taille du train dataset : 61828
Taille du test dataset : 3254


In [ ]:
import random

# Sélectionner un index aléatoire
random_index = random.randint(0, len(train_dataset) - 1)

# Récupérer le datapoint aléatoire
random_datapoint = train_dataset[random_index]

# Décoder et afficher le vers original
decoded_verse = tokenizer.decode(random_datapoint['input_ids'].tolist())
print("\nVers décodé:")
print(decoded_verse.strip())  # strip() pour enlever les PAD à la fin

# Afficher quelques statistiques
print("\nStatistiques:")
print(f"Longueur du vers: {len(decoded_verse.strip())} caractères")
print(f"Nombre de tokens: {len(random_datapoint['input_ids'])}")

# Afficher les 10 premiers caractères du vocabulaire (en excluant <PAD> et <UNK>)
print("\nExemple de caractères dans le vocabulaire:")
print(list(tokenizer.vocab.keys())[2:12])


Vers décodé:
Oui, de ce jour fatal, plein d'horreur et de charmes,<PAD><PAD><PAD><PAD><PAD><PAD><PAD>

Statistiques:
Longueur du vers: 88 caractères
Nombre de tokens: 60

Exemple de caractères dans le vocabulaire:
['R', 'e', 'c', 'u', 'i', 'l', ' ', ':', 'L', 's']


## Entraînement du modèle

In [ ]:
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_positions=60,
    n_ctx=60,
    n_embd=128,
    n_layer=8,
    n_head=8,
    bos_token_id=tokenizer.vocab['<PAD>'],
    eos_token_id=tokenizer.vocab['<PAD>'],
    pad_token_id=tokenizer.vocab['<PAD>']
)

model = GPT2LMHeadModel(config)

In [ ]:
from transformers import TrainingArguments, Trainer

# Définition des hyperparamètres de base
num_train_epochs = 10
learning_rate = 1e-4
batch_size = 8

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./char-level-gpt-french-poetry-verses",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=0.01,
    logging_steps=100,
    save_steps=1000,
    eval_steps=500,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

# Configuration du Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Lancement de l'entraînement
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
500,1.636400,1.594355
1000,1.532200,1.494720
1500,1.475000,1.429542
2000,1.410700,1.381300
2500,1.368800,1.333717
3000,1.344700,1.305961
3500,1.349400,1.280853
4000,1.295100,1.260016
4500,1.301300,1.242583
5000,1.283500,1.215278


KeyboardInterrupt: 

In [ ]:
# Sauvegarde du modèle
trainer.save_model()

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
from huggingface_hub import HfApi, HfFolder
import shutil
import os

# Chemin vers le checkpoint que vous voulez sauvegarder
checkpoint_path = "/content/char-level-gpt-french-poetry-verses/checkpoint-40000"

# Chemin où vous voulez sauvegarder le modèle final
final_model_path = "/content/final_model"

# Créez le répertoire pour le modèle final s'il n'existe pas
os.makedirs(final_model_path, exist_ok=True)

# Copiez les fichiers nécessaires
files_to_copy = ['config.json', 'model.safetensors', 'generation_config.json']
for file in files_to_copy:
    shutil.copy(os.path.join(checkpoint_path, file), final_model_path)

# Chargez le modèle à partir du checkpoint
config = AutoConfig.from_pretrained(checkpoint_path)
model = AutoModelForCausalLM.from_pretrained(checkpoint_path, config=config)

# Sauvegardez le modèle dans le nouveau répertoire
model.save_pretrained(final_model_path)

# Définissez le nom du dépôt sur Hugging Face où vous voulez charger le modèle
repo_name = "Zual/poetGPT"

# Chargez le modèle sur Hugging Face
model.push_to_hub(repo_name)

print(f"Le modèle final (checkpoint-40000) a été sauvegardé et chargé sur {repo_name}")

model.safetensors:   0%|          | 0.00/6.44M [00:00<?, ?B/s]

Le modèle final (checkpoint-40000) a été sauvegardé et chargé sur Zual/poetGPT


## Si vous avez la flemme de réentraîner le modèle depuis le début, vous pouvez charger celui que j'ai entraîné

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Remplacez par le nom de votre dépôt sur Hugging Face
repo_name = "Zual/poetGPT"

# Chargez le modèle
model = AutoModelForCausalLM.from_pretrained(repo_name)

print(f"Le modèle et le tokenizer ont été chargés depuis {repo_name}")

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.44M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Le modèle et le tokenizer ont été chargés depuis Zual/poetGPT


## Générer des vers a partir d'un préfixe ! Et acrostiches

In [ ]:
def generate_verse(prompt, max_length=60):
    input_ids = torch.tensor([tokenizer.encode(prompt)]).to(model.device)
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                            no_repeat_ngram_size=2, do_sample=True, top_k=50, top_p=0.20,
                            pad_token_id=tokenizer.vocab['<PAD>'])
    return tokenizer.decode(output[0].tolist())

# Exemple de génération
prompt = "A"
generated_verse = generate_verse(prompt)
print(f"Vers généré : {generated_verse}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Vers généré : Au souffle des francheurs,<PAD>


In [ ]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_verse(prompt, max_length=60):
    input_ids = torch.tensor([tokenizer.encode(prompt)]).to(model.device)
    output = model.generate(input_ids, max_length=max_length, num_return_sequences=1,
                            no_repeat_ngram_size=2, do_sample=True, top_k=50, top_p=0.20,
                            pad_token_id=tokenizer.vocab['<PAD>'])
    return tokenizer.decode(output[0].tolist())

acrostiche = 'PoetGPT'

print("Génération de vers pour chaque lettre de l'acrostiche :")
for letter in acrostiche:
    generated_verse = generate_verse(letter)
    print(f"{generated_verse}")

Génération de vers pour chaque lettre de l'acrostiche :
Pour le coeul des flots, son branche,<PAD>
ou de l'amor qui s'en voit aux parfums comme<PAD>
et de la main enfante, ondamne<PAD>
te commence, et les flots,<PAD>
Grand de la main et sonte, au forme<PAD>
Pour le coeul des flots, son branche,<PAD>
Tout ce qui sont, en voilà la paix des fleurs,<PAD>
